In [2]:
### Standard Imports - Sorry PEP8 fans, do not look below
import pandas as pd, numpy as np, os, re, json, sys
from pathlib import Path
from datetime import datetime
from pathlib import Path
import os
import numpy as np
import pandas as pd

import random
from joshberry.utils import *
from joshberry.feats import *

## Specific Imports

In [3]:
### set path directories
curr_dir = Path(os.getcwd())
data_dir = Path(curr_dir.parents[0] / 'data/')
artifacts_dir = Path(curr_dir.parents[0] / 'artifacts/')

### Helper Functions

In [10]:
def add_new_rows(data):
    orig_rows = len(data)
    data['simulate_n'] = np.random.poisson(0.6, orig_rows)
    distinct_n = data.simulate_n.unique()
    
    result=[]
    for n in distinct_n:
        temp = df[df['simulate_n']==n]
        
        for j in range(1,n+1):
            if len(result) == 0:
                result = temp
            else:
                result = pd.concat([result, temp])
            
    return result.reset_index(drop=True)

def random_date(x):
    s_d = pd.to_datetime(x[x['start_date'].notna()]['start_date'])
    e_d = pd.to_datetime(x[x['end_date'].notna()]['end_date'])
    
    start_u = s_d.iloc[0].value//10**9
    end_u = e_d.iloc[0].value//10**9
    try:
        end_date_list = pd.to_datetime(np.random.randint(start_u, end_u, len(x)-1), unit='s').values
    except:
        print(x)
    end_date_list = np.append(end_date_list, e_d.values)
    x['end_date'] = end_date_list
    mask = x['start_date'].isna()
    x.loc[mask,'start_date'] = x.shift(1).loc[mask]['end_date'].astype(str)
    x['start_date'] = pd.to_datetime(x['start_date']).dt.date
    x['end_date'] = pd.to_datetime(x['end_date']).dt.date
    return x

In [11]:
seed = 20171230
np.random.seed(seed)
output_filename = 'customer_billing.csv'

In [12]:
df = pd.read_csv(Path(data_dir) / 'customer_account.csv')
df = df[df['account_status']=='ACTIVE'][['AcctID','start_date','end_date']]

for dtcol in ['start_date','end_date']:
    df[dtcol] =pd.to_datetime(df[dtcol])
    
df['days'] = df['end_date'] - df['start_date']















df

,AcctID,start_date,end_date,days
0,C1000089533,2015-06-04,2020-04-09,1771 days
2,C1000410568,2016-02-28,2016-04-09,41 days
4,C1000410568,2016-10-09,2021-04-01,1635 days
5,C1000585011,2015-04-16,2018-08-03,1205 days
7,C1000585011,2020-07-13,2021-04-01,262 days
...,...,...,...,...
10095,C992743972,2015-05-17,2021-04-01,2146 days
10096,C99460026,2015-06-12,2021-04-01,2120 days
10097,C995808151,2015-05-29,2021-04-01,2134 days
10098,C998796548,2015-07-10,2021-04-01,2092 days
